In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/teo/userdata/SejongMusic


In [2]:
import torch
from typing import *
from omegaconf import OmegaConf

from sejong_music.constants import  POSITION, PITCH, PART
from sejong_music.jg_code import JeongganDataset, JeongganTokenizer, JeongganPiece
from sejong_music.model_zoo import JeongganTransSeq2seq
from sejong_music.inference import JGInferencer
from sejong_music.jg_to_staff_converter import JGToStaffConverter


/home/teo/.local/share/virtualenvs/SejongMusic-0Xd-sAL0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = JeongganDataset(use_offset=True)


AttributeError: 'NoneType' object has no attribute 'pos'

In [4]:
piece = JeongganPiece('music_score/gen_code/관악영산회상 가락덜이_daegeum_piri_haegeum_ajaeng.txt', use_offset=True)
piece.sliced_parts_by_measure[0]['daegeum'][:5]

['beat:0', '청무', '노네', '|', 'beat:2/3']

In [5]:
idx = 100

piece = JeongganPiece('music_score/gen_code/여민락_daegeum_piri_haegeum_ajaeng_gayageum_geomungo.txt', use_offset=True)
tokens = piece.sliced_parts_by_measure[idx]['daegeum']

piece_b = JeongganPiece('music_score/gen_code/여민락_daegeum_piri_haegeum_ajaeng_gayageum_geomungo.txt', use_offset=False)
tokens_b = piece_b.sliced_parts_by_measure[idx]['daegeum']

for a, b in zip(tokens, tokens_b):
  print(a, b)

beat:0 :2
청태 청태
beat:1/3 :5
청중 청중
| |
beat:1/3 :5
- -
겹흘림표 겹흘림표
beat:2/3 :8
청태 청태
| |
beat:0 :0
청황 청황
| |
beat:0 :0
청황 청황
니나 니나
| |
| |
beat:0 :2
청황 청황
니나 니나
beat:2/3 :8
청태 청태
니레 니레
| |
beat:0 :2
청황 청황
beat:2/3 :8
같은음표 같은음표
니나 니나
| |
beat:0 :2
청태 청태
beat:1/2 :6
니나* 니나*
beat:2/3 :8
청황 청황
| |
beat:0 :0
청중 청중
| |
beat:2/3 :8
청태 청태

 

beat:0 :0
청중 청중
| |
| |
beat:0 :0
청태 청태
| |
beat:0 :0
청태 청태
니레 니레
| |
| |
beat:0 :0
청태 청태
니레 니레
| |
beat:1/2 :6
노 노
beat:2/3 :8
청중 청중
| |
beat:0 :2
청남 청남
노네 노네
beat:2/3 :8
청임 청임
| |
beat:0 :0
청중 청중
| |
beat:1/3 :5
- -
미는표 미는표
beat:2/3 :8
청태 청태

 

beat:0 :0
청중 청중
| |
| |
beat:0 :0
청태 청태
| |
beat:1/2 :6
니나* 니나*
beat:2/3 :8
청황 청황
| |
beat:0 :0
청중 청중
| |
beat:0 :0
청태 청태
| |
beat:0 :2
청중 청중
beat:2/3 :8
청태 청태
| |
beat:0 :2
청중 청중
beat:1/3 :5
- -
미는표 미는표
beat:2/3 :8
청태 청태
| |
beat:0 :0
청황 청황
| |
beat:0 :0
같은음표 같은음표
니나 니나

 

beat:0 :0
청중 청중
| |
beat:0 :2
청태 청태
beat:1/2 :6
니나* 니나*
beat:2/3 :8
청황 청황
| |
beat:0 :0
청중 청중
| |
beat:0 :0
청남 청남
노네 노네
| |
beat:0 :2
같은음표 같은음

In [6]:
from sejong_music.jg_code import ABCPiece
from sejong_music.jg_to_staff_converter import ABCtoGenConverter

abc_piece = ABCPiece('music_score/gen_code/관악영산회상 가락덜이_daegeum_piri_haegeum_ajaeng.txt')
abc_tokens = abc_piece.sliced_parts_by_measure[4]['ajaeng']
abc_tokens = [x[0] for x in abc_tokens]

gen_tokens = piece.sliced_parts_by_measure[4]['ajaeng']

conv_gen_tokens = ABCtoGenConverter()(abc_tokens).split(' ')
conv_gen_tokens = [x for x in conv_gen_tokens if x != '']
for conv, org in zip(conv_gen_tokens, gen_tokens):
    print(conv, org)

num_jg is 10
num_jg is 10
num_jg is 10
num_jg is 10
:0 beat:0
임 태
| beat:1/3
| 중
:0 |
무 |
미는표 beat:0
| 태
:0 |
청황 |
| |
| beat:0
:0 태
임 |
| beat:0
| 중
:0 |
무 beat:0
미는표 태
| |
:0 |
협 |
| beat:0

 중
:0 미는표
임 |
| beat:0
| 임
:0 |
임 beat:0
| 중
:0 미는표
청황 |
| beat:0
| 황
:0 |
임 beat:0
| 태
| beat:2/3
:0 중
청황 |
| beat:0
:0 남
무 |
| beat:0

 중
:0 |
중 |
| |
| 

:0 beat:0
황 태
| beat:1/3
:0 중
임 |
| |
:0 beat:0
중 태
| |
:0 |
황 |
| beat:0
:0 태
협 |
| beat:0
:0 중
중 |
| beat:0
:0 태
중 |
| |

 |
:0 beat:0
무 중
| |
| beat:0
:0 중
임 |
| beat:0
:0 남
청황 |
| beat:0
| 남
:0 |
임 beat:0
| 임
| |
:0 beat:0
무 임
미는표 |
| beat:0
:0 중
협 |
| |


In [7]:
note_in_measure[-1].offset

NameError: name 'note_in_measure' is not defined

In [8]:
self = ABCtoGenConverter()
tokens = abc_tokens

notes, _ = self.decoder.convert_abc_tokens(tokens)
note_by_measure = self.group_by_attribute(notes, 'gak_offset')

for i, note_in_measure in enumerate(note_by_measure):
  conv_jgs = self.to_omr_converter.list_of_abc_notes_to_jeonggan(note_in_measure)
  text_jgs = self.to_omr_converter.jeonggan_note_to_text(conv_jgs)
  print(len(conv_jgs))
  # part_text.append('|'.join(text_jgs))


num_jg is 10
10
num_jg is 10
10
num_jg is 10
10
num_jg is 10
10


In [9]:
import music21

abc_note, abc_score = JGToStaffConverter().convert_abc_tokens(abc_tokens)
gen_note, gen_score = JGToStaffConverter()(gen_tokens)

score = music21.stream.Score()
score.insert(0, abc_score)
score.insert(0, gen_score)

score.show()

Unknown pos None, beat:0, ['황'], 79


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [9]:
dataset = JeongganDataset(use_offset=True)
decoded = dataset.tokenizer.decode(dataset[1][0])

In [ ]:
decoded = dataset.tokenizer.decode(dataset[3][2][:-1])
tokens = decoded
tokens

In [ ]:
from fractions import Fraction
from sejong_music.jg_to_staff_converter import ABCtoGenConverter
from sejong_music.constants import BEAT_POSITION
from sejong_music.abc_utils import ABCNote

total_notes = []
global_jg_offset = 0
prev_notes = []
prev_position = 0
position_tokens = BEAT_POSITION
gak_idx = 0
jg_idx = 0
for token in tokens:
  if token in ('|', '\n'):      
    global_jg_offset += 1
    if prev_notes:
      total_notes.append(ABCNote(prev_notes, duration=global_jg_offset-prev_position, global_offset=prev_position, jg_offset=jg_idx, gak_offset=gak_idx))
      prev_notes = []
    if token == '|': 
      jg_idx += 1
    if token == '\n':
      gak_idx += 1
      jg_idx = 0
    continue
  if token in BEAT_POSITION:
    current_offset = global_jg_offset + Fraction(token[5:])
    if prev_notes:
      total_notes.append(ABCNote(prev_notes, duration=current_offset-prev_position, global_offset=prev_position, jg_offset=jg_idx, gak_offset=gak_idx))
    prev_position = current_offset
    prev_notes = []
  else:
    prev_notes.append(token)

converter = ABCtoGenConverter()
note_by_measure = converter.group_by_attribute(total_notes, 'gak_offset')
note_by_measure = [x for x in note_by_measure if len(x)>0]
part_text = []
for i, note_in_measure in enumerate(note_by_measure):
  conv_jgs = converter.to_omr_converter.list_of_abc_notes_to_jeonggan(note_in_measure)
  text_jgs = converter.to_omr_converter.jeonggan_note_to_text(conv_jgs)
  part_text.append('|'.join(text_jgs))
gen_code = converter.gencode_converter.convert_lines_to_gencode(part_text)
gen_code_tokens = [x for x in gen_code.split(' ') if x != '']

for gen, off_code in zip(gen_code_tokens, tokens):
  
  print(gen , off_code)

In [45]:
def get_piece_and_part(dataset, name, idx):
  for piece in dataset.all_pieces:
    for i, part in enumerate(piece.tokenized_parts):
      if piece.name == name and i == idx:
        return piece, part
dataset = JeongganDataset(use_offset=True, is_valid=False)

piece, part = get_piece_and_part(dataset, '관악영산회상 상현도드리', 1)

In [82]:
jg_tokens = piece.parts[1].split(' ')
jg_tokens = [x for x in jg_tokens if x != '']
jg_tokens = piece.filter_by_ignore_token(jg_tokens)
jgs = ' '.join(jg_tokens).replace('\n', '|').split('|')
beat_jgs = ' '.join(piece.tokenized_parts[1]).replace('\n', '|').split('|')
jgs = [x.replace('\n', '') for x in jgs if x.strip() != '']
beat_jgs = [x.replace('\n', '') for x in beat_jgs if x.strip() != '']
for jg, beat_jg in zip(jgs, beat_jgs):
  print(f"{jg} | {beat_jg}")

:0 임  | beat:0 임 
 :0 중  |  beat:0 중 
 :2 태 :6 노 :8 중  |  beat:0 태 beat:1/2 노 beat:2/3 중 
 :0 태  |  beat:0 태 
 :1 임 :3 청황 :8 남 서침표  |  beat:0 임 beat:1/6 청황 beat:2/3 남 서침표 
 :10 임 :11 겹요성표  |  beat:0 임 beat:1/2 겹요성표 
 :1 임 :3 청황 :5 - 미는표 :8 임  |  beat:0 임 beat:1/6 청황 beat:1/3 - 미는표 beat:2/3 임 
 :0 중  |  beat:0 중 
 :8 쉼표  |  beat:2/3 쉼표 
 :1 태 :3 중  |  beat:0 태 beat:1/6 중 
 :0 - 미는표  |  beat:0 - 미는표 
 :0 중  |  beat:0 중 
 :8 배남  |  beat:2/3 배남 
 :0 태 나니로  |  beat:0 태 나니로 
 :6 노 :8 중  |  beat:1/2 노 beat:2/3 중 
 :1 배남 :3 배임 :8 배남  |  beat:0 배남 beat:1/6 배임 beat:2/3 배남 
 :2 태 :9 황  |  beat:0 태 beat:5/6 황 
 :0 중  |  beat:0 중 
 :0 태  |  beat:0 태 
 :2 황 느니라 :6 니 :8 배남  |  beat:0 황 느니라 beat:1/2 니 beat:2/3 배남 
 :0 배임  |  beat:0 배임 
 :0 배임 니나  |  beat:0 배임 니나 
 :2 배임 :8 배남 :9 노  |  beat:0 배임 beat:2/3 배남 beat:5/6 노 
 :0 태 미는표  |  beat:0 태 미는표 
 :2 배남 :8 배임  |  beat:0 배남 beat:2/3 배임 
 :2 황 :8 니나*  |  beat:0 황 beat:2/3 니나* 
 :0 배임 시루표  |  beat:0 배임 시루표 
 :0 배임 니나  |  beat:0 배임 니나 
 :2 배남 :6 배임 :8 배남  

In [3]:
import re
entire_positions = []
dataset = JeongganDataset(use_offset=True, is_valid=False)
for j, piece in enumerate(dataset.all_pieces):
  for i, part in enumerate(piece.parts):
    jg_tokens = part.split(' ')
    jg_tokens = [x for x in jg_tokens if x != '']
    # jg_tokens = piece.filter_by_ignore_token(jg_tokens)
    # split by both '|' and '\n'
    jgs = ' '.join(jg_tokens).replace('\n', '|').split('|') 
    pattern = r":(\d+)"
    for jg in jgs:
      matches = re.findall(pattern, jg)
      matches = tuple([int(x) for x in matches])
      if matches == (2,5,6,8):
        print(piece.name, i, jg)
      entire_positions.append(matches)


counter = Counter(entire_positions)

남창우조 중거 0  :2 황 미는표 :5 니 :6 배남 :8 배남 나니로 
남창우조 중거 0  :2 황 미는표 :5 니 :6 배남 :8 배남 나니로 
남창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 중거 0  :2 황 미는표 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 니나 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 중거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 평거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 평거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 평거 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 평거 0  :2 황 미는표 :5 니 :6 배남 :8 배남 나니로 
남창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
자진한잎 경풍년 변조두거 0  :2 청황 :5 니 :6 남 :8 남 나니로 
자진한잎 경풍년 변조두거 0  :2 청황 :5 니 :6 남 :8 남 나니로 
남창계면 평롱 0  :2 황 니레 :5 니나* :6 중 :8 무 느니-르 
여창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
여창우조 이수대엽 0  :2 황 :5 니 :6 배남 :8 배남 나니로 
남창계면 언롱 0  :2 황 :5 니나* :6 중 :8 무 느니-르 
여창

In [81]:
counter.most_common()

[((0,), 66574),
 ((), 27619),
 ((2, 8), 9126),
 ((8,), 3455),
 ((2, 6, 8), 2592),
 ((2, 5, 8), 2152),
 ((2, 5), 1749),
 ((11,), 1412),
 ((2, 9), 1085),
 ((2,), 1072),
 ((10, 11), 835),
 ((6, 8), 823),
 ((5, 8), 800),
 ((9,), 704),
 ((1, 3), 702),
 ((1, 3, 8), 596),
 ((5, 9), 583),
 ((15,), 493),
 ((10, 15), 347),
 ((2, 6, 7, 9), 251),
 ((1, 5), 209),
 ((1, 3, 9), 209),
 ((2, 5, 9), 146),
 ((1, 3, 5), 141),
 ((2, 7, 9), 124),
 ((1, 3, 5, 8), 97),
 ((2, 8, 9), 89),
 ((7, 9), 85),
 ((1, 3, 6, 8), 69),
 ((6, 7, 9), 62),
 ((1, 3, 5, 9), 55),
 ((5,), 47),
 ((14, 15), 46),
 ((2, 5, 6, 8), 36),
 ((5, 8, 9), 35),
 ((2, 5, 8, 9), 28),
 ((1, 5, 9), 21),
 ((1, 2, 3, 5), 20),
 ((12, 13, 11), 16),
 ((8, 9), 14),
 ((12, 13), 9),
 ((1, 3, 4, 6, 8), 8),
 ((2, 6, 7, 8, 9), 8),
 ((2, 5, 7, 9), 8),
 ((1, 3, 7, 9), 7),
 ((1, 3, 8, 9), 6),
 ((5, 7, 9), 6),
 ((1, 3, 5, 7, 9), 6),
 ((1, 3, 6, 7, 9), 6),
 ((2, 5, 6, 7, 9), 4),
 ((2, 7, 8, 9), 3),
 ((12, 13, 14, 15), 3),
 ((2, 4, 6, 7, 9), 2),
 ((1, 2, 3, 8), 2

In [73]:
valid_positions = [ (0,),(2,5), (2, 8), (2,7,9), ()]

import re
self=piece
jg_tokens = self.parts[1].split(' ')
jg_tokens = [x for x in jg_tokens if x != '']
jg_tokens = self.filter_by_ignore_token(jg_tokens)
jgs = ' '.join(jg_tokens).split('|')
pattern = r":(\d+)"
for jg in jgs:
  matches = re.findall(pattern, jg)
  matches = tuple([int(x) for x in matches])


(2, 5)
()
(8,)
(2, 8, 9)
(0,)
(2, 8)
(0,)
(8,)
(0,)
(0,)
(5, 9)
(0,)
()
(5, 9)
(0,)
(0,)
(0,)
(2, 6, 8)
(0,)
(2, 8)
(0, 0)
(0,)
(0,)
(0,)
()
(5, 9)
(0,)
()
(0,)
(0,)
(2, 8, 5, 9)
(0,)
()
(0,)
(2, 5)
(0,)
()
()
()
()
(0, 0)
(5, 8)
(0,)
(0,)
()
(5, 9)
(0,)
(0,)
(8, 9)
(0,)
(0,)
(8, 9)
(0,)
(0,)
(0,)
(5, 8, 9, 0)
(0,)
(5, 8, 9)
(0,)
(8,)
(0, 0)
(5, 8, 9)
(0,)
(5, 9)
(0,)
(0, 2, 9)
(0,)
(5, 9)
(2, 8)
(0,)
(8, 0)
(0,)
(0,)
(8,)
(0,)
(8, 0)
(0,)
()
(0,)
(0,)
(0, 0)
(0,)
(2, 5)
(0,)
(8,)
(0, 0)
(8,)
(0,)
(5, 9)
(0,)
(8, 0)
(5, 8)
(0,)
(0,)
(0,)
(8, 0)
()
(2, 9)
(0,)
(2, 8)
(0, 0)
(9,)
(0,)
(5, 9)
(0,)
(0,)
()
()
()
()
(0, 0)
(2, 5)
()
(0,)
()
(8, 0)
(0,)
(0,)
()
(0,)
(2, 6, 8, 0)
(5, 9)
(0,)
(0,)
(0,)
(2, 8, 0)
(0,)
()
(0,)
(0,)
(0, 5, 9)
(0,)
()
(0,)
(5, 8, 9)
(0, 0)
(5, 8, 9)
(0,)
(0,)
(0,)
(5, 9)
(0,)
()
(5, 9)
(0,)
(0,)
(0,)
(2, 6, 8)
(0,)
(2, 8)
(0, 0)
(0,)
(0,)
(0,)
()
(5, 9)
(0,)
()
(0,)
(0,)
(2, 8, 5, 9)
(0,)
()
(0,)
(2, 5)
(0,)
()
()
()
()
(0, 0)
(5, 8)
(0,)
(0,)
()
()


In [63]:
for jg, beat_jg in zip(jgs, beat_jgs):
  print(f"{jg}")

:2 중 :5 임 서침표 
 :8 니나* 
 :2 중 :8 임 :9 노 
 :0 청황 
  :2 임 서침표 :8 니나* 
 :0 중 
 :8 니나* 
 :0 협 
 :0 쉼표 
  :5 임 :9 노 
 :0 청황 
 :5 임 서침표 :9 노 
 :0 청황 
  :0 임 서침표 
 :0 중 
 :2 임 서침표 :6 중 :8 협 
 :0 황 
 :2 협 :8 황 
 :0 중  :0 임 
 :0 - 미는표 
 :0 협 
 :0 쉼표 
  :5 임 :9 노 
 :0 청황 
 :0 청황 노니로 
 :0 무 미는표 
 :2 협 :8 황  :5 임 :9 노 
 :0 청황 
 :0 임 서침표 
 :2 협 미는표 :5 중 
  :0 임 서침표 
 :0 쉼표  :0 임 
 :5 - 미는표 :8 황 시루표 
 :0 협 
 :0 쉼표 
  :5 임 :9 노 
 :0 무 
 :0 무 
 루러표 :8 임 :9 노 
 :0 청황 
  :0 무 
 루러표 :8 임 :9 노 
 :0 청황 
 :0 무 
 :0 협 
 :5 - 미는표 :8 임 :9 노  :0 무 
 :0 임 서침표 
 :5 - 루러표 :8 임 :9 노 
 :0 청황 
 :8 임 서침표 
 :0 무  :0 협 
 :5 - 미는표 :8 임 :9 노 
 :0 무 
 :5 임 서침표 :9 노 
 :0 무 
 :0 쉼표  :2 배임 :9 배임 니나 
 :0 황 
 :5 협 :9 노 
 :2 임 :8 니나* 
 :0 중 
 :8 황 시루표  :0 협 
 :0 중 
 :0 임 서침표 
 :8 임 서침표 
 :0 중 
 :8 황 시루표  :0 협 
 :0 임 
 :0 임 미는표 
 :0 협 
 :0 쉼표  :0 황 
 :0 협 
 :2 황 :5 임 
 :0 중 
 :8 황 시루표 
 :0 중  :0 임 
 :8 중 
 :0 황 시루표 
 :5 협 :9 노 
 :0 중 
 :8 황 시루표  :0 임 
 :5 - 미는표 :8 황 
 :0 중 
 :0 임 서침표 
 :0 중 
 :8 황 시루표  :0 협 
 :2 배임 :9 배임 니나 
 :0 

In [62]:
jg_tokens

[':2',
 '중',
 ':5',
 '임',
 '서침표',
 '|',
 '|',
 ':8',
 '니나*',
 '|',
 ':2',
 '중',
 ':8',
 '임',
 ':9',
 '노',
 '|',
 ':0',
 '청황',
 '|',
 '\n',
 ':2',
 '임',
 '서침표',
 ':8',
 '니나*',
 '|',
 ':0',
 '중',
 '|',
 ':8',
 '니나*',
 '|',
 ':0',
 '협',
 '|',
 ':0',
 '쉼표',
 '|',
 '\n',
 ':5',
 '임',
 ':9',
 '노',
 '|',
 ':0',
 '청황',
 '|',
 '|',
 ':5',
 '임',
 '서침표',
 ':9',
 '노',
 '|',
 ':0',
 '청황',
 '|',
 '\n',
 ':0',
 '임',
 '서침표',
 '|',
 ':0',
 '중',
 '|',
 ':2',
 '임',
 '서침표',
 ':6',
 '중',
 ':8',
 '협',
 '|',
 ':0',
 '황',
 '|',
 ':2',
 '협',
 ':8',
 '황',
 '|',
 ':0',
 '중',
 '\n',
 ':0',
 '임',
 '|',
 ':0',
 '-',
 '미는표',
 '|',
 ':0',
 '협',
 '|',
 ':0',
 '쉼표',
 '|',
 '|',
 '\n',
 ':5',
 '임',
 ':9',
 '노',
 '|',
 ':0',
 '청황',
 '|',
 '|',
 ':0',
 '청황',
 '노니로',
 '|',
 ':0',
 '무',
 '미는표',
 '|',
 ':2',
 '협',
 ':8',
 '황',
 '\n',
 ':5',
 '임',
 ':9',
 '노',
 '|',
 ':0',
 '청황',
 '|',
 '|',
 ':0',
 '임',
 '서침표',
 '|',
 ':2',
 '협',
 '미는표',
 ':5',
 '중',
 '|',
 '\n',
 ':0',
 '임',
 '서침표',
 '|',
 '|',
 '|',
 '|',
 '|',
 ':0',
 '쉼표

In [35]:
from sejong_music.abc_utils import convert_beat_jg_to_gen

dataset = JeongganDataset(use_offset=True, is_valid=False)
# tokens = dataset.all_pieces[0].tokenized_parts[0]
for piece in dataset.all_pieces:
  for i, part in enumerate(piece.tokenized_parts):
    print(piece.name, i)
    
    conv_tokens = convert_beat_jg_to_gen(part)
    gen_tokens = piece.split_and_filter(piece.parts[i])
    for conv, gen in zip(conv_tokens, gen_tokens):
      if conv != gen:
        # break all for loop
        print(piece.name, i)
        break
      

영산회상 상령산 0
영산회상 상령산 1
영산회상 상령산 2
영산회상 상령산 3
영산회상 상령산 4
자진한잎 수룡음 편1 0
자진한잎 수룡음 편1 1
자진한잎 수룡음 편1 2
자진한잎 수룡음 편1 3
자진한잎 수룡음 편1 4
자진한잎 수룡음 편1 5
천년만세 우조가락도드리 0
천년만세 우조가락도드리 1
천년만세 우조가락도드리 2
천년만세 우조가락도드리 3


AssertionError: gen_tokens not resolved: [<sejong_music.abc_utils.BeatToken object at 0x7fde1cb7bfd0>, '황', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7beb0>, '하배황', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b2e0>, '배남', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7ba00>, '배남', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7bdf0>, '배임', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b190>, '배임', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7bc10>, '하배황', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7bf70>, '배임', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b460>, '배임', '\n', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b4c0>, '배중', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b430>, '배태', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b580>, '배황', '|', <sejong_music.abc_utils.BeatToken object at 0x7fde1cb7b160>, '배태', '|', <sejong_music.abc_ut

In [31]:
piece.parts[0]

':0 임 니나 |  | :0 중 흘림표 | :2 태 :6 노 :8 중 | :2 태 너녜 :8 쉼표 \n :1 임 :3 청황 :6 니 :8 남 | :2 임 :6 니 :8 느나 반길이표/덧길이표 | :1 임 :3 청황 :6 니 :8 임 | :2 중 :5 쉼표 :8 중 미는표 |  |  |  |  |  |  | :8 쉼표 | :2 태 :5 임 |  | :0 중 흘림표 | :0 태 | :9 배남 \n :0 태 나니로 | :6 노 :8 중 | :1 배남 :3 배임 :8 쉼표 | :2 태 :9 노 | :0 중 흘림표 | :0 태 | :2 황 니나 :6 니 :8 배남 | :0 배임 | :0 배임 니나 |  | :2 배임 니나 :8 배남 니레 | :2 태 :6 니 :7 태 :9 노 | :2 배남 :6 니나* :8 배임 | :2 황 :8 니레나 | :2 배남 흘림표 :5 배임 니나 | :0 배임 니나 \n :2 배남 :6 노 :8 배남 | :0 태 | :2 황 니나 :6 니 :7 배남 :9 배임 | :2 태 :9 노 | :0 중 흘림표 | :0 태 | :2 황 니나 :6 니 :8 배남 | :0 배임 | :0 배임 니나 |  | :2 태 :6 노 :8 중 | :1 배남 :3 황 |  |  | :8 니 | :2 노 :6 니 :8 느나 \n :0 중 | :0 - 풀어내림표 | :0 태 너녜 | :2 황 니나 :6 태 :8 배남 | :0 배임 | :2 황 :5 니 :6 배남 :8 배남 나니로 | :0 태 나니로 | :8 쉼표 | :2 태 :5 중 | :5 - 흘림표 :8 태 나느나 | :2 배남 :8 황 | :0 태 니레 |  \n :2 황 니나 :6 태 :8 배남 | :0 배임 |  | :2 황 :5 니 :6 배남 :8 배남 나니로 | :0 태 나니로 |  |  |  |  |  | :8 쉼표 | :2 황 니나 :6 니 :8 배남 | :0 배임 |  |  | :8 쉼표 \n :2 태 :5 중 | :8 임 나니나 | :0 중 | :2 황 :7 태 니레 :9 황 | :0 중 풀어내림표

In [12]:
dataset = JeongganDataset(use_offset=True, is_valid=True)
decoded = dataset.tokenizer.decode(dataset[10][2])
tokens = decoded

beat_tokens = []
prev_resolved = True
for token in tokens:
  if token[:4] == 'beat':
    if not prev_resolved:
      if beat_tokens[-1] == 'beat:0':
        if token == 'beat:1/2':
          beat_tokens[-1] = ':10'
          prev_resolved = True
        if token == 'beat:2/3':
          beat_tokens[-1] = ':2'
          prev_resolved = True
        elif token == 'beat:1/3':
          beat_tokens[-1] = ':2'
          prev_resolved = True
        elif token == 'beat:1/6':
          beat_tokens[-1] = ':1'
          prev_resolved = True
        elif token == 'beat:5/6':
          beat_tokens[-1] = ':2'
          prev_resolved = True
        elif token == 'beat:3/4':
          beat_tokens[-1] = ':10'
          prev_resolved = True
      elif beat_tokens[-1] == 'beat:1/6':
          beat_tokens[-1] = ':3'
          prev_resolved = True
      elif beat_tokens[-1] == 'beat:1/3':
        if token in ('beat:2/3', 'beat:5/6'):
          beat_tokens[-1] = ':5'
          prev_resolved = True
        elif token == 'beat:1/2':
          beat_tokens[-1] = ':4'
          prev_resolved = True
      elif beat_tokens[-1] == 'beat:1/2':
        if token == 'beat:2/3':
          beat_tokens[-1] = ':6'
          prev_resolved = True
      elif beat_tokens[-1] == 'beat:2/3':
        if token == 'beat:5/6':
          beat_tokens[-1] = ':7'
          prev_resolved = True
    beat_tokens.append(token)
    prev_resolved = False
  if token.strip() in ('|' '\n'):
    # can calculate token duration
    if prev_resolved:
      continue
    if beat_tokens[-1] == 'beat:0':
      beat_tokens[-1] = ':0'
      prev_resolved = True
    elif beat_tokens[-1] == 'beat:1/6':
      beat_tokens[-1] = ':1'
      prev_resolved = True
    elif beat_tokens[-1] == 'beat:1/3':
      beat_tokens[-1] = ':5'
      prev_resolved = True
    elif beat_tokens[-1] == 'beat:2/3':
      beat_tokens[-1] = ':8'
      prev_resolved = True
    elif beat_tokens[-1] == 'beat:5/6':
      beat_tokens[-1] = ':9'
      prev_resolved = True
    elif beat_tokens[-1] == 'beat:1/2':
      if beat_tokens[-2] in (':0', ':10', ':11'):
        beat_tokens[-1] = ':11'
        prev_resolved = True
      elif beat_tokens[-2] in (':2', ':3', ':4'):
        beat_tokens[-1] = ':6'
        prev_resolved = True
    elif beat_tokens[-1] == 'beat:3/4':
      beat_tokens[-1] = ':15'
      prev_resolved = True
    
beat_tokens

[':2',
 ':8',
 ':2',
 ':9',
 ':0',
 ':2',
 ':5',
 ':2',
 ':8',
 ':2',
 ':8',
 ':2',
 ':5',
 ':8',
 ':1',
 ':3',
 ':5',
 ':2',
 ':5',
 ':8',
 ':8',
 ':1',
 ':1',
 ':10',
 ':6',
 ':8',
 ':10',
 ':6',
 ':8',
 ':1',
 ':3',
 ':8',
 ':1',
 ':3',
 ':5',
 ':8',
 ':6',
 ':8',
 ':2',
 ':8',
 ':1',
 ':3',
 ':8',
 ':2',
 ':8',
 ':10',
 ':6',
 ':8',
 ':10',
 ':6',
 ':7',
 ':9',
 ':2',
 ':5',
 ':8',
 ':6',
 ':8',
 ':0',
 ':8',
 ':0',
 ':1',
 ':1',
 ':2',
 ':8',
 ':0',
 ':2',
 ':8',
 ':2',
 ':8',
 ':10',
 ':6',
 ':8',
 ':0',
 ':1',
 ':3',
 ':9',
 ':2',
 ':8',
 ':0',
 ':2',
 ':8',
 ':2',
 ':8',
 ':2',
 ':8',
 ':0',
 ':9']

In [ ]:
import numpy as np
from sejong_music.jg_to_staff_converter import Note


def convert_tokens_to_roll(tokens:List[str], inst:str, num_frame_per_jg=6, num_features=6)->np.ndarray:
  notes:List[Note] = JGToStaffConverter.convert_to_notes(tokens)
  JGToStaffConverter.get_duration_of_notes(notes)
  num_jgs = sum([1 for x in tokens if x in ('|', '\n')])

  outputs = np.zeros((num_jgs * num_frame_per_jg, num_features), dtype=object)
  outputs[:, 2] = [f'beat:{i}' for i in range(6)] * num_jgs
  outputs[:, 5] = inst
  num_jg_per_gak = JeongganPiece.get_measure_length(tokens)
  cur_idx = 0
  for i, num_jg in enumerate(num_jg_per_gak):
    outputs[cur_idx:cur_idx+num_jg*num_frame_per_jg, 3] = [f'jg:{i}' for i in np.arange(num_jg).repeat(num_frame_per_jg)]
    outputs[cur_idx:cur_idx+num_jg*num_frame_per_jg, 4] = f'gak:{i}'
    cur_idx += num_jg*num_frame_per_jg

  for note in notes:
    jg_offset, beat_offset, duration = note.global_jg_offset, note.beat_offset, note.duration
    if duration is None:
      duration = 0
    start_frame = round((jg_offset + beat_offset) * num_frame_per_jg)
    end_frame = max(round((jg_offset + beat_offset + duration) * num_frame_per_jg), start_frame+1)
    outputs[start_frame, 0] = note.pitch
    outputs[start_frame+1:end_frame, 0] = '-'
    if note.ornaments:
      outputs[start_frame, 1] = note.ornaments[0]
  #   outputs[start_frame:end_frame, 0] = note.pitch
  #   if note.ornaments:
  #     outputs[start_frame, 1] = note.ornaments[0]
  #   else:
  #     outputs[start_frame, 1] = '음표시작'
  outputs[outputs==0] = '비어있음'
  return outputs

In [ ]:
tokens = piece.tokenized_parts[0]
notes:List[Note] = JGToStaffConverter.convert_to_notes(tokens)
JGToStaffConverter._fix_three_col_division(notes)
for note in notes:
  print(note.gak_offset, note.jg_offset, note.beat_offset)

In [ ]:
notes[note_id-1].beat_offset

In [ ]:
notes[-1].beat_offset

In [ ]:
from sejong_music.constants import POSITION

note_id = 0
new_tokens = []

for token in tokens:
  if token in POSITION[2:]:
    new_token = notes[note_id].beat_offset
    new_tokens.append(new_token)
    note_id += 1
  else:
    new_tokens.append(token)
new_tokens